# Transformer를 이용한 감정분석기 - 한국어 Data

제작자: Park Chanjun (박찬준) <br>
소속: Korea University Natural Language Processing & Artificial Intelligence Lab<br>
Email: bcj1210@naver.com<br>
참고자료: https://keras.io/


# 데이터 전처리

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME='C:\Program Files\Java\jdk-14.0.1'

In [ ]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential # 케라스의 Sequential()을 임포트
from tensorflow.keras.layers import Dense # 케라스의 Dense()를 임포트
from tensorflow.keras import optimizers # 케라스의 옵티마이저를 임포트
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import SimpleRNN, LSTM, Embedding, Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Dropout
from keras.optimizers import SGD,Adam
from keras.layers import BatchNormalization
from keras.layers import Activation
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
path = '/content/drive/Shareddrives/인공지능 경진대회/감성분류/input'
with open(path+'/'+'train_data.json', 'r') as f:
    json_train = json.load(f)
with open(path+'/'+'dev_data.json', 'r') as f:
    json_dev = json.load(f)

In [ ]:
#사용할 데이터만 분류
#talk-id, type, content 빼와서 새로운 리스트 형성
train_data = []
for case in json_train:
    add_case = dict()
    add_case['type'] = case['profile']['emotion']['type'] #emotion type
    add_case['content'] = case['talk']['content'] #content
    add_case['talk-id'] = case['talk']['id']['talk-id'] #talk-id
    train_data.append(add_case)

dev_data = []
for case in json_dev:
    add_case = dict()
    add_case['type'] = case['profile']['emotion']['type'] #emotion type
    add_case['content'] = case['talk']['content'] #content
    add_case['talk-id'] = case['talk']['id']['talk-id'] #talk-id
    dev_data.append(add_case)

#자료 확인
dev_data[0]

{'content': {'HS01': '노후준비 하기가 정말 어렵네.',
  'HS02': '돈이 제일 문제야.',
  'HS03': '적금이나 저축을 하긴 했는데 얼마 안되니까 문제지.',
  'SS01': '제일 걱정이 뭐세요?',
  'SS02': '지금 노후준비 하고 계신건가요?',
  'SS03': '저축을 해도 금액적으로 액수가 적은것 같다고 생각하시는군요.'},
 'talk-id': 'Pro_08728_00002',
 'type': 'E37'}

In [ ]:
#content의 value를 string 형태로 변환
datalist = [train_data, dev_data]
for l in datalist:
  for dic in l:
    dialogue = dic['content']
    dia_string = ''
    for value in dialogue.values():
      dia_string += value + ' '
    dic['content'] = dia_string

#자료형 확인
train_data[0]

{'content': '기분이 좋지 않아 기분이 좋지 않은 이유를 알 수 있을까요? 요새 건강검진을 받았는데 건강 상태가 좋지 않아. 많이 속상하시겠어요. 어디가 안 좋은가요? 이곳 저곳 관리해야 할 곳이 많아. 안좋은 곳이 많아 걱정이 많으시겠어요. ',
 'talk-id': 'Pro_10888_00001',
 'type': 'E37'}

In [ ]:
train_df = pd.DataFrame.from_dict(train_data)
train_df.head()

,type,content,talk-id
0,E37,기분이 좋지 않아 기분이 좋지 않은 이유를 알 수 있을까요? 요새 건강검진을 받았는...,Pro_10888_00001
1,E37,진로가 너무 걱정스러워. 내가 진짜 뭘 해야 할지 모르겠어. 미래가 두려우시군요. ...,Pro_06568_00003
2,E37,아이가 친구랑 다퉈서 걱정이야. 걱정이 되시겠군요. 아이는 어떤 상태인가요? 아이도...,Pro_10528_00004
3,E37,나 요새 친구들이랑 대화하는 게 어려워. 친구들과 대화하는 것이 어려운 이유가 있을...,Pro_06208_00005
4,E37,결혼 후 맞벌이를 하고 있는데 은퇴를 생각하면 어렵네. 은퇴를 생각하며 걱정이 되시...,Pro_08728_00006


In [ ]:
dev_df = pd.DataFrame.from_dict(dev_data)
dev_df.head()

,type,content,talk-id
0,E37,노후준비 하기가 정말 어렵네. 제일 걱정이 뭐세요? 돈이 제일 문제야. 지금 노후준...,Pro_08728_00002
1,E37,나이가 드니까 건강이 걱정스러워. 친한 친구가 암에 걸렸거든. 친구가 암에 걸려서 ...,Pro_10888_00017
2,E37,남자친구랑 십 년 가까이 만났는데 결혼적령기가 되니 결혼해도 될지 고민되네. 결혼을...,Pro_06208_00018
3,E37,걱정이 하나 있어. 어떤 걱정인지 여쭤봐도 될까요? 다들 새로운 친구를 잘 사귀는데...,Pro_06208_00021
4,E37,요즘 젊은 직원들은 도대체 이해할 수 없어 직원들과 무슨일이 있으셨나봐요. 왜이렇게...,Pro_09208_00038


In [ ]:
X_train = []
X_test = []

tokenizer = Tokenizer()

for sentence in train_df['content']:
  X_train.append([word for word in str(sentence).split(" ")])

for sentence in dev_df['content']:
  X_test.append([word for word in str(sentence).split(" ")])

tokenizer.fit_on_texts(X_train) # 단어 인덱스를 구축합니다.
tokenizer.fit_on_texts(X_test) # 단어 인덱스를 구축합니다.

X_train = tokenizer.texts_to_sequences(X_train) # 문자열을 정수 인덱스의 리스트로 변환합니다.
X_test = tokenizer.texts_to_sequences(X_test) # 문자열을 정수 인덱스의 리스트로 변환합니다.


maxlen = 128  # 128 
vocab_size = len(tokenizer.word_index)+1 # 단어의 수

X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen) # 패딩
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen) # 패딩

## 소분류

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

labels_train = lb_make.fit_transform(train_df['type'].values) # string에서 int로 변환
labels_train = to_categorical(labels_train) # int에서 one-hot encoding

labels_test = lb_make.fit_transform(dev_df['type'].values) # string에서 int로 변환
labels_test = to_categorical(labels_test) # int에서 one-hot encoding

## 대분류

In [ ]:
# y_train, y_dev 불러오기
y_train = []; y_dev = []

y_train = train_df['type'].values; y_dev = dev_df['type'].values

In [ ]:
# 대분류

y_train_big = []
y_dev_big = []

regex1 = re.compile('E1.'); regex2 = re.compile('E2.'); regex3 = re.compile('E3.') # 정규 표현식 
regex4 = re.compile('E4.'); regex5 = re.compile('E5.'); regex6 = re.compile('E6.')

for i in range(len(y_train)):
    matchobj1 = regex1.match(y_train[i]);matchobj2 = regex2.match(y_train[i]);matchobj3 = regex3.match(y_train[i]);
    matchobj4 = regex4.match(y_train[i]);matchobj5 = regex5.match(y_train[i]);matchobj6 = regex6.match(y_train[i]);
    if matchobj1 != None:
        y_train_big.append(0)
    elif matchobj2 != None:
        y_train_big.append(1)
    elif matchobj3 != None:
        y_train_big.append(2)
    elif matchobj4 != None:
        y_train_big.append(3)   
    elif matchobj5 != None:
        y_train_big.append(4)
    elif matchobj6 != None:
        y_train_big.append(5)
    else:
        pass

for i in range(len(y_dev)):
    matchobj1 = regex1.match(y_dev[i]);matchobj2 = regex2.match(y_dev[i]);matchobj3 = regex3.match(y_dev[i]);
    matchobj4 = regex4.match(y_dev[i]);matchobj5 = regex5.match(y_dev[i]);matchobj6 = regex6.match(y_dev[i]);
    if matchobj1 != None:
        y_dev_big.append(0)
    elif matchobj2 != None:
        y_dev_big.append(1)
    elif matchobj3 != None:
        y_dev_big.append(2)
    elif matchobj4 != None:
        y_dev_big.append(3)   
    elif matchobj5 != None:
        y_dev_big.append(4)
    elif matchobj6 != None:
        y_dev_big.append(5)
    else:
        pass

labels_train_big = to_categorical(y_train_big) # int에서 one-hot encoding
labels_test_big = to_categorical(y_dev_big) # int에서 one-hot encoding

## Transformer 모델

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs) #Multihead Attn 블록
        attn_output = self.dropout1(attn_output, training=training) #드롭아웃
        out1 = self.layernorm1(inputs + attn_output) #LM + Residual
        ffn_output = self.ffn(out1) #FF 블록
        ffn_output = self.dropout2(ffn_output, training=training) #드롭아웃
        return self.layernorm2(out1 + ffn_output) #LM + Residual

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emded_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=emded_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=emded_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1) #포지션 정보
        positions = self.pos_emb(positions) #포지션 임베딩
        x = self.token_emb(x) #토큰임베딩
        return x + positions #합치기

In [ ]:
embed_dim = 32  # Embedding size for each token(원래 32)
num_heads = 2  # Number of attention heads(원래 2)
ff_dim = 32  # Hidden layer size in feed forward network inside transformer(원래 32)

inputs = layers.Input(shape=(maxlen,)) #처음 입력
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim) #객체 생성
x = embedding_layer(inputs)  #포지셔널 임베딩
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim) #객체 생성
x = transformer_block(x) #트랜스포머 
x = layers.GlobalAveragePooling1D()(x) #Average Pooling
x = layers.Dropout(0.2)(x) #드롯아웃
x = layers.Dense(256, activation="relu")(x) #FFNN
x = layers.Dropout(0.2)(x) #드롭아웃
outputs = layers.Dense(60, activation="softmax")(x) #Softmax

model = keras.Model(inputs=inputs, outputs=outputs) #모델 생성

In [ ]:
embed_dim = 32  # Embedding size for each token(원래 32)
num_heads = 2  # Number of attention heads(원래 2)
ff_dim = 32  # Hidden layer size in feed forward network inside transformer(원래 32)

inputs = layers.Input(shape=(maxlen,)) #처음 입력
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim) #객체 생성
x = embedding_layer(inputs)  #포지셔널 임베딩
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim) #객체 생성
x = transformer_block(x) #트랜스포머 
x = layers.GlobalAveragePooling1D()(x) #Average Pooling
x = layers.Dropout(0.2)(x) #드롯아웃
x = layers.Dense(20, activation="relu")(x) #FFNN
x = layers.Dropout(0.2)(x) #드롭아웃
outputs = layers.Dense(6, activation="softmax")(x) #Softmax

model2 = keras.Model(inputs=inputs, outputs=outputs) #모델 생성

## 학습

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

LR_INIT = 2e-4
EXP_DECAY = 0.9

def lr_schedule_fn(epoch):
    num = 30
    if epoch < num:
        lr = LR_INIT
    else:
        lr = LR_INIT * EXP_DECAY**(epoch-num)
    return lr

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule_fn)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(LR_INIT), loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(X_train, labels_train, batch_size=32, epochs=50, 
                    callbacks=[es, lr_callback],validation_split=0.1)

Epoch 1/50
446/446 [==============================] - 12s 27ms/step - loss: 4.1030 - acc: 0.0171 - val_loss: 4.1257 - val_acc: 0.0152
Epoch 2/50
446/446 [==============================] - 12s 27ms/step - loss: 4.0744 - acc: 0.0293 - val_loss: 4.1069 - val_acc: 0.0278
Epoch 3/50
446/446 [==============================] - 12s 27ms/step - loss: 3.9309 - acc: 0.0577 - val_loss: 3.9881 - val_acc: 0.0499
Epoch 4/50
446/446 [==============================] - 12s 27ms/step - loss: 3.4255 - acc: 0.1421 - val_loss: 3.6571 - val_acc: 0.1509
Epoch 5/50
446/446 [==============================] - 12s 27ms/step - loss: 2.4406 - acc: 0.3685 - val_loss: 3.5931 - val_acc: 0.2891
Epoch 6/50
446/446 [==============================] - 12s 27ms/step - loss: 1.4784 - acc: 0.6237 - val_loss: 3.9735 - val_acc: 0.3782
Epoch 7/50
446/446 [==============================] - 12s 27ms/step - loss: 0.8238 - acc: 0.8030 - val_loss: 4.5602 - val_acc: 0.3920
Epoch 8/50
446/446 [==============================] - 12s 27ms

In [ ]:
model2.compile(optimizer=tf.keras.optimizers.Adam(LR_INIT), loss='categorical_crossentropy', metrics=['acc'])

history = model2.fit(X_train, labels_train_big, batch_size=32, epochs=50, 
                    callbacks=[es, lr_callback],validation_split=0.1)

Epoch 1/50
446/446 [==============================] - 13s 29ms/step - loss: 1.7976 - acc: 0.1761 - val_loss: 1.7902 - val_acc: 0.1932
Epoch 2/50
446/446 [==============================] - 13s 28ms/step - loss: 1.7381 - acc: 0.2482 - val_loss: 1.6784 - val_acc: 0.3131
Epoch 3/50
446/446 [==============================] - 12s 28ms/step - loss: 1.4862 - acc: 0.4020 - val_loss: 1.5503 - val_acc: 0.4028
Epoch 4/50
446/446 [==============================] - 13s 28ms/step - loss: 1.1441 - acc: 0.5599 - val_loss: 1.5229 - val_acc: 0.4646
Epoch 5/50
446/446 [==============================] - 13s 28ms/step - loss: 0.7997 - acc: 0.7033 - val_loss: 1.6754 - val_acc: 0.5025
Epoch 6/50
446/446 [==============================] - 13s 28ms/step - loss: 0.5430 - acc: 0.8133 - val_loss: 1.9543 - val_acc: 0.5032
Epoch 7/50
446/446 [==============================] - 13s 29ms/step - loss: 0.3515 - acc: 0.8884 - val_loss: 2.1367 - val_acc: 0.5455
Epoch 8/50
446/446 [==============================] - 13s 29ms

## 성능 측정

In [ ]:
#모델 정보 출력
model.summary() 

#성능 측정
test_loss,test_acc=model.evaluate(X_test,labels_test)
print("Test_acc: ",test_acc)

Model: "functional_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 128)]             0         
_________________________________________________________________
token_and_position_embedding (None, 128, 32)           2562752   
_________________________________________________________________
transformer_block_17 (Transf (None, 128, 32)           6464      
_________________________________________________________________
global_average_pooling1d_16  (None, 32)                0         
_________________________________________________________________
dropout_66 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_134 (Dense)            (None, 256)               8448      
_________________________________________________________________
dropout_67 (Dropout)         (None, 256)             

In [ ]:
#모델 정보 출력
model.summary() 

#성능 측정
test_loss,test_acc=model.evaluate(X_test,labels_test)
print("Test_acc: ",test_acc)

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
token_and_position_embedding (None, 128, 128)          10251008  
_________________________________________________________________
transformer_block_4 (Transfo (None, 128, 128)          132480    
_________________________________________________________________
global_average_pooling1d_3 ( (None, 128)               0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 256)               33024     
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)              

In [ ]:
#모델 정보 출력
model2.summary() 

#성능 측정
test_loss,test_acc=model2.evaluate(X_test,labels_test_big)
print("Test_acc: ",test_acc)

Model: "functional_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 128)]             0         
_________________________________________________________________
token_and_position_embedding (None, 128, 32)           2562752   
_________________________________________________________________
transformer_block_11 (Transf (None, 128, 32)           6464      
_________________________________________________________________
global_average_pooling1d_10  (None, 32)                0         
_________________________________________________________________
dropout_42 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_86 (Dense)             (None, 20)                660       
_________________________________________________________________
dropout_43 (Dropout)         (None, 20)              

In [ ]:
#모델 정보 출력
model2.summary() 

#성능 측정
test_loss,test_acc=model2.evaluate(X_test,labels_test_big)
print("Test_acc: ",test_acc)

Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 128)]             0         
_________________________________________________________________
token_and_position_embedding (None, 128, 32)           2562752   
_________________________________________________________________
transformer_block_16 (Transf (None, 128, 32)           6464      
_________________________________________________________________
global_average_pooling1d_15  (None, 32)                0         
_________________________________________________________________
dropout_62 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_126 (Dense)            (None, 20)                660       
_________________________________________________________________
dropout_63 (Dropout)         (None, 20)              